In [13]:
from __future__ import division

import warnings
from collections import namedtuple
import torch
import torch.nn as nn
from torch.jit.annotations import Optional, Tuple
from torch import Tensor
import os
import numpy as np
import os.path
from glob import glob
from PIL import Image
from tqdm import tqdm
import torchvision.datasets as dset
import torch.utils.data as data
from torchvision import transforms
from ipywidgets import IntProgress
import logging 
import matplotlib.pyplot as plt

In [14]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else: 
    device = 'cpu'
print(device)

cuda


In [15]:
learning_rate=0.001
batch_train_size=64
batch_test_size=64
num_epochs=150

In [16]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
])

In [17]:
train_dataset = dset.CIFAR10(root='./data', train=True, transform=train_transform, download=True)
test_dataset = dset.CIFAR10(root='./data', train=False, transform=test_transform, download=True)


train_loader = data.DataLoader(dataset=train_dataset, batch_size=batch_train_size, shuffle=True)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=batch_test_size, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [18]:
class Inception(nn.Module):
    def __init__(self, in_planes, kernel_1_x, kernel_3_in, kernel_3_x, kernel_5_in, kernel_5_x, pool_planes):
        super(Inception, self).__init__()

      
        self.b1 = nn.Sequential(
            nn.Conv2d(in_planes, kernel_1_x, kernel_size=1),
            nn.BatchNorm2d(kernel_1_x),
            nn.ReLU(True),
        )

     
        self.b2 = nn.Sequential(
            nn.Conv2d(in_planes, kernel_3_in, kernel_size=1),
            nn.BatchNorm2d(kernel_3_in),
            nn.ReLU(True),
            nn.Conv2d(kernel_3_in, kernel_3_x, kernel_size=3, padding=1),
            nn.BatchNorm2d(kernel_3_x),
            nn.ReLU(True),
        )

     
        self.b3 = nn.Sequential(
            nn.Conv2d(in_planes, kernel_5_in, kernel_size=1),
            nn.BatchNorm2d(kernel_5_in),
            nn.ReLU(True),
            nn.Conv2d(kernel_5_in, kernel_5_x, kernel_size=5, padding=2),
            nn.BatchNorm2d(kernel_5_x),
            nn.ReLU(True),
        )

       
        self.b4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_planes, pool_planes, kernel_size=1),
            nn.BatchNorm2d(pool_planes),
            nn.ReLU(True),
        )

    def forward(self, x):
        out1 = self.b1(x)
        out2 = self.b2(x)
        out3 = self.b3(x)
        out4 = self.b4(x)
        return torch.cat([out1, out2, out3, out4], 1)



In [19]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        
        self.conv1 = nn.Conv2d(3, 192, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(192)
        self.relu1 = nn.ReLU(inplace=True)
        
       
        self.inception1a = Inception(192, 64, 96, 128, 16, 32, 32)  # outputs 256 channels
        self.inception1b = Inception(256, 128, 128, 192, 32, 96, 64)  # outputs 480 channels
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        
        self.inception2a = Inception(480, 192, 96, 208, 16, 48, 64)  # outputs 512 channels
        self.inception2b = Inception(512, 160, 112, 224, 24, 64, 64)  # outputs 512 channels
        self.inception2c = Inception(512, 128, 128, 256, 24, 64, 64)  # outputs 512 channels
        self.inception2d = Inception(512, 112, 144, 288, 32, 64, 64)  # outputs 528 channels
        self.inception2e = Inception(528, 256, 160, 320, 32, 128, 128)  # outputs 832 channels
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
      
        self.inception3a = Inception(832, 256, 160, 320, 32, 128, 128)  # outputs 832 channels
        self.inception3b = Inception(832, 384, 192, 384, 48, 128, 128)  # outputs 1024 channels
        
    
        self.avgpool = nn.AvgPool2d(kernel_size=8)  # Global average pooling (8x8 feature maps)
        self.fc = nn.Linear(1024, 10)  # Final fully connected layer
        
    def forward(self, x):
        # Initial layer
        x = self.relu1(self.bn1(self.conv1(x)))
        
 
        x = self.inception1a(x)
        x = self.inception1b(x)
        x = self.maxpool1(x)
        
      
        x = self.inception2a(x)
        x = self.inception2b(x)
        x = self.inception2c(x)
        x = self.inception2d(x)
        x = self.inception2e(x)
        x = self.maxpool2(x)
        
      
        x = self.inception3a(x)
        x = self.inception3b(x)
        
        
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        
        return x

model = CNN().to(device)


In [20]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50, 100], gamma=0.5)
criterion = nn.CrossEntropyLoss()

In [ ]:
train_losses, test_losses = [], []
train_accuracies, test_accuracies = [], []

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_accuracy = 100. * correct / total
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    
    scheduler.step()

    
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    test_accuracy = 100. * correct / total
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

    print(f"Epoch [{epoch+1}/{num_epochs}]\n"
          f"Train Accuracy: {train_accuracy:.2f}% | Train Loss: {train_loss:.4f}\n"
          f"Test Accuracy: {test_accuracy:.2f}% | Test Loss: {test_loss:.4f}")


torch.save(model.state_dict(), 'cnn_cifar10.pth')


plt.figure()
plt.plot(range(num_epochs), train_losses, label='Train Loss')
plt.plot(range(num_epochs), test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss over Epochs')
plt.show()


Epoch [1/150]
Train Accuracy: 53.81% | Train Loss: 983.0477
Test Accuracy: 65.61% | Test Loss: 153.4278
Epoch [2/150]
Train Accuracy: 72.22% | Train Loss: 614.1575
Test Accuracy: 70.06% | Test Loss: 129.2599
Epoch [3/150]
Train Accuracy: 78.05% | Train Loss: 493.5321
Test Accuracy: 74.57% | Test Loss: 115.9805
Epoch [4/150]
Train Accuracy: 81.33% | Train Loss: 423.0254
Test Accuracy: 74.75% | Test Loss: 121.4535
Epoch [5/150]
Train Accuracy: 83.20% | Train Loss: 377.6206
Test Accuracy: 80.97% | Test Loss: 87.6588
Epoch [6/150]
Train Accuracy: 85.31% | Train Loss: 333.9218
Test Accuracy: 80.79% | Test Loss: 88.4614
Epoch [7/150]
Train Accuracy: 86.68% | Train Loss: 306.5041
Test Accuracy: 84.50% | Test Loss: 69.8143
Epoch [8/150]
Train Accuracy: 87.54% | Train Loss: 283.1038
Test Accuracy: 83.35% | Test Loss: 79.9531
Epoch [9/150]
Train Accuracy: 88.77% | Train Loss: 258.7033
Test Accuracy: 85.42% | Test Loss: 67.4199
Epoch [10/150]
Train Accuracy: 89.45% | Train Loss: 240.9136
Test Acc